In [1]:
from functools import partial, reduce
from os import rename, makedirs
import os
import urllib.request
import pandas as pd
import os.path
import shutil
import json
import csv
import re
import datetime
from typing import Literal

In [ ]:
# carpeta = f"../output/INFDES1/definitivos"

# csv_files = [x for x in os.listdir(carpeta) if ".csv" in x]

# for f in csv_files: 
#     new = f"{carpeta}/{f.replace('_old','')}"
#     old = f"{carpeta}/{f}"
#     os.rename(old, new)


In [2]:
def gsheet_download_csv(id: str, target: str, url_template="https://docs.google.com/spreadsheets/d/{id}/export?format=csv"):
    url = url_template.format(id=id)
    return urllib.request.urlretrieve(url, target)

gsheet_download_csv('1UyfAnRCes0OAOUR0aARfkfegTsUl4Puw', 'nomenclador.csv')
nomenclador = pd.read_csv('./nomenclador.csv')
pais = {k:v for k,v in nomenclador[['codigo_fundar', 'desc_fundar']].iloc}
continentes = {k:v for k,v in nomenclador[['codigo_fundar', 'continente_fundar']].iloc}
nivel_agregacion = {k:v for k,v in nomenclador[['codigo_fundar', 'nivel_agregacion']].iloc}
codes_set = set(pais.keys())

lista_cambios = []
diccionario_cambios = {}

In [3]:
def match_relocate(src, target, pattern, file_list=None):
    files_to_move = file_list or os.listdir(src)

    pattern = re.compile(pattern)

    if not os.path.exists(target):
        makedirs(target)

    for filename in files_to_move:
        if pattern.match(filename):
            src_path = os.path.join(src, filename)
            dest_path = os.path.join(target, filename)

            shutil.move(src_path, dest_path)

    print(f"Files matching the pattern '{pattern}' moved to '{target}'.")

def compose2(f, g):
    def composed(*a, **kw):
        return f(g(*a, **kw))

    return composed

def compose(*fs):
    return reduce(compose2, fs)

def _chain(*fs):
    return compose(*reversed(fs))

def chain(*fs):
    lista_cambios.clear()
    return _chain(*fs)

def exportar_definitivo(archivo: str, folder:str, df: pd.DataFrame, nuevo_nombre=None):
    if not nuevo_nombre:
        nuevo_nombre = f"{folder}/definitivos/{archivo}_old.csv"

    original_file = f'{folder}/definitivos/{archivo}.csv'

    rename(original_file, nuevo_nombre)

    df.to_csv(original_file, 
              encoding='utf-8', 
              sep=',', 
              quoting=csv.QUOTE_ALL, 
              quotechar='"', 
              lineterminator='\n', 
              decimal='.', 
              index=False)
    
    return original_file, nuevo_nombre

def drop_col(df: pd.DataFrame, col, axis=1, lista_cambios = lista_cambios):
    lista_cambios.append(f"Se borró la columna {col}")
    return df.drop(col, axis=axis)

def drop_colx(col, axis=1):
    return lambda df: drop_col(df, col, axis=axis)

def normalize_countries(df: pd.DataFrame, code_col:str, new_col:str, desc_col:str, map: dict, lista_cambios = lista_cambios):
    codes_set = set(map.keys())
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    diff_set = set(df[code_col].dropna()) - codes_set
    if len(diff_set)>0: 
        diff_str = ", ".join([str(x) for x in list(diff_set)])
        string = f"ERROR: Los códigos {diff_str} no se encontraron en el nomenclador vigente al {today} en la columna {code_col}"
        lista_cambios.append(string)
    else:
        if (new_col and desc_col):
            df[new_col] = df[code_col].apply(map.__getitem__)
            df = df.drop(desc_col, axis=1)
            string = f"Usando los valores de la columna {code_col}, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, creando la columna {new_col} y borrando la columna {desc_col}"
        elif desc_col:
            df[desc_col] = df[code_col].apply(map.__getitem__)
            string = f"Usando los valores de la columna {code_col}, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, cambiando los valores de la columna {desc_col}"
        else: 
            string = f"Usando los valores de la columna {code_col}, se verificó que todos los códigos se encuentran en el nomenclador vigente al {today}"
        lista_cambios.append(string)
    return df

def normalize_countriesx(code_col, new_col, desc_col, map):
    return lambda df: normalize_countries(df, code_col=code_col, new_col=new_col, desc_col=desc_col, map=map)

def wide_to_long(df: pd.DataFrame, primary_keys, value_name='valor', var_name='indicador', lista_cambios=lista_cambios):
    cols = ", ".join(df.columns)
    new_cols = ", ".join(primary_keys + [value_name, var_name])
    string = f"Long estricto: [{cols}] -> [{new_cols}]"
    lista_cambios.append(string)
    return df.melt(id_vars=primary_keys, value_name=value_name, var_name=var_name)

def wide_to_longx(primary_keys, value_name='valor', var_name='indicador'):
    return lambda df: wide_to_long(df, primary_keys, value_name=value_name, var_name=var_name)

def rename_cols(df: pd.DataFrame, map, lista_cambios=lista_cambios):
    map_string = ", ".join(f"{old_col}:{new_col}" for old_col, new_col in map.items())
    string = f"Se renombraron las columnas: {map_string}"
    lista_cambios.append(string)
    df = df.rename(columns=map)
    return df

def rename_colsx(map):
    return lambda df: rename_cols(df, map)

def replace_value(df:pd.DataFrame, col:str, curr_value:str, new_value:str, lista_cambios=lista_cambios):
    string = f"Se imputó el valor {new_value} en la columna {col} cuando {col}=={curr_value}"
    lista_cambios.append(string)
    df = df.replace({col: curr_value}, new_value)
    return df

def replace_valuex(col, curr_value, new_value):
    return lambda df: replace_value(df=df, col=col, curr_value=curr_value, new_value=new_value)

def sort_vals_asc(df:pd.DataFrame, prim_keys:list, lista_cambios=lista_cambios):
    lista_cambios.append(f"Ordené las filas por las columnas: {', '.join(prim_keys)}")
    return df.sort_values(by=prim_keys).reset_index(drop=True)

def sort_vals_ascx(prim_keys):
    return lambda df: sort_vals_asc(df=df, prim_keys=prim_keys)

def cast_col(df:pd.DataFrame, col:str, cast_to:Literal['int','float','object'], lista_cambios=lista_cambios):
    old_type = str(df[col].dtype)
    df[col] = df[col].astype(cast_to)
    lista_cambios.append(f"El tipo de dato de la columna {col} fue cambiado {old_type} ->> {cast_to}")
    return df

def cast_colx(col, cast_to):
    return lambda df: cast_col(df=df, col=col, cast_to=cast_to)

CRECIM_g1.csv

In [4]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 1

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,iso3,pais_nombre,continente_fundar,es_agregacion,anio,pib,pib_pc
0,ABW,Aruba,"América del Norte, Central y el Caribe",0,1986,405600000.0,6283.00
1,ABW,Aruba,"América del Norte, Central y el Caribe",0,1987,487700000.0,7567.25
2,ABW,Aruba,"América del Norte, Central y el Caribe",0,1988,596600000.0,9274.51
3,ABW,Aruba,"América del Norte, Central y el Caribe",0,1989,695500000.0,10767.40
4,ABW,Aruba,"América del Norte, Central y el Caribe",0,1990,764800000.0,11638.73


In [14]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
nomenclar_continentes = partial(normalize_countriesx, map=continentes)
nomenclar_nivel_agg = partial(normalize_countriesx, map=nivel_agregacion)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)


pipeline = chain(
    nomenclar_nivel_agg(code_col="iso3", desc_col="es_agregacion", new_col="nivel_agregacion"),
    nomenclar_paises(code_col='iso3', desc_col="pais_nombre", new_col=None), 
    nomenclar_continentes(code_col="iso3", desc_col="continente_fundar", new_col=None),
    to_long(primary_keys + ['pais_nombre','continente_fundar','nivel_agregacion']),
    sort_vals_ascx(primary_keys)
    
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'region_code': 'cod_area'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)


diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,pais_nombre,continente_fundar,nivel_agregacion,indicador,valor
0,ABW,1986,Aruba,"América del Norte, Central y el Caribe",pais,pib,4.056000e+08
1,ABW,1986,Aruba,"América del Norte, Central y el Caribe",pais,pib_pc,6.283000e+03
2,ABW,1987,Aruba,"América del Norte, Central y el Caribe",pais,pib,4.877000e+08
3,ABW,1987,Aruba,"América del Norte, Central y el Caribe",pais,pib_pc,7.567250e+03
4,ABW,1988,Aruba,"América del Norte, Central y el Caribe",pais,pib,5.966000e+08


In [15]:
diccionario_cambios

{'CRECIM_g1': ['Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-25, creando la columna nivel_agregacion y borrando la columna es_agregacion',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-25, cambiando los valores de la columna pais_nombre',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-25, cambiando los valores de la columna continente_fundar',
  'Long estricto: [iso3, pais_nombre, continente_fundar, anio, pib, pib_pc, nivel_agregacion] -> [iso3, anio, pais_nombre, continente_fundar, nivel_agregacion, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio']}

In [18]:
exportar(_df)

('../output/CRECIM1/definitivos/CRECIM_g1.csv',
 '../output/CRECIM1/definitivos/CRECIM_g1_old.csv')

CRECIM_g2.csv

In [19]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 2

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)

df.head()

iso3 anio


,iso3,pais_nombre,continente_fundar,es_agregacion,anio,expectativa_al_nacer,pib_pc
0,ABW,Aruba,"América del Norte, Central y el Caribe",0,1990,73.08,30823.48
1,ABW,Aruba,"América del Norte, Central y el Caribe",0,1991,73.10,32222.66
2,ABW,Aruba,"América del Norte, Central y el Caribe",0,1992,73.18,32986.54
3,ABW,Aruba,"América del Norte, Central y el Caribe",0,1993,73.23,34336.55
4,ABW,Aruba,"América del Norte, Central y el Caribe",0,1994,73.27,35984.83


In [17]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
nomenclar_continentes = partial(normalize_countriesx, map=continentes)
nomenclar_nivel_agg = partial(normalize_countriesx, map=nivel_agregacion)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)


pipeline = chain(
    nomenclar_nivel_agg(code_col="iso3", desc_col="es_agregacion", new_col="nivel_agregacion"),
    nomenclar_paises(code_col='iso3', desc_col="pais_nombre", new_col=None), 
    nomenclar_continentes(code_col="iso3", desc_col="continente_fundar", new_col=None),
    to_long(primary_keys + ['pais_nombre','continente_fundar','nivel_agregacion']),
    sort_vals_ascx(primary_keys)
    
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    renombrar({'_valor': 'valor'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)


diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,pais_nombre,continente_fundar,nivel_agregacion,indicador,valor
0,ABW,1990,Aruba,"América del Norte, Central y el Caribe",pais,expectativa_al_nacer,73.08
1,ABW,1990,Aruba,"América del Norte, Central y el Caribe",pais,pib_pc,30823.48
2,ABW,1991,Aruba,"América del Norte, Central y el Caribe",pais,expectativa_al_nacer,73.10
3,ABW,1991,Aruba,"América del Norte, Central y el Caribe",pais,pib_pc,32222.66
4,ABW,1992,Aruba,"América del Norte, Central y el Caribe",pais,expectativa_al_nacer,73.18


In [18]:
diccionario_cambios

{'CRECIM_g1': ['Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-25, creando la columna nivel_agregacion y borrando la columna es_agregacion',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-25, cambiando los valores de la columna pais_nombre',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-25, cambiando los valores de la columna continente_fundar',
  'Long estricto: [iso3, pais_nombre, continente_fundar, anio, pib, pib_pc, nivel_agregacion] -> [iso3, anio, pais_nombre, continente_fundar, nivel_agregacion, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio'],
 'CRECIM_g2': ['Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-25, creando la columna niv

In [19]:
exportar(_df)

('../output/CRECIM1/definitivos/CRECIM_g2.csv',
 '../output/CRECIM1/definitivos/CRECIM_g2_old.csv')

CRECIM_g3.csv

In [20]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 3

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,iso3,pais_nombre,continente_fundar,es_agregacion,anio,pib_percapita_ppp_2017,anios_escol_prom
0,AFG,Afganistán,Asia,0,2002,1280.46,1.52
1,AFG,Afganistán,Asia,0,2003,1292.33,1.58
2,AFG,Afganistán,Asia,0,2004,1260.06,1.64
3,AFG,Afganistán,Asia,0,2005,1352.32,1.69
4,AFG,Afganistán,Asia,0,2006,1366.99,1.78


In [21]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
nomenclar_continentes = partial(normalize_countriesx, map=continentes)
nomenclar_nivel_agg = partial(normalize_countriesx, map=nivel_agregacion)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)


pipeline = chain(
    nomenclar_nivel_agg(code_col="iso3", desc_col="es_agregacion", new_col="nivel_agregacion"),
    nomenclar_paises(code_col='iso3', desc_col="pais_nombre", new_col=None), 
    nomenclar_continentes(code_col="iso3", desc_col="continente_fundar", new_col=None),
    to_long(primary_keys + ['pais_nombre','continente_fundar','nivel_agregacion']),
    sort_vals_ascx(primary_keys)
    
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'region_code': 'cod_area'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)


diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,pais_nombre,continente_fundar,nivel_agregacion,indicador,valor
0,AFG,2002,Afganistán,Asia,pais,pib_percapita_ppp_2017,1280.46
1,AFG,2002,Afganistán,Asia,pais,anios_escol_prom,1.52
2,AFG,2003,Afganistán,Asia,pais,pib_percapita_ppp_2017,1292.33
3,AFG,2003,Afganistán,Asia,pais,anios_escol_prom,1.58
4,AFG,2004,Afganistán,Asia,pais,pib_percapita_ppp_2017,1260.06


In [22]:
diccionario_cambios

{'CRECIM_g1': ['Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-25, creando la columna nivel_agregacion y borrando la columna es_agregacion',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-25, cambiando los valores de la columna pais_nombre',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-25, cambiando los valores de la columna continente_fundar',
  'Long estricto: [iso3, pais_nombre, continente_fundar, anio, pib, pib_pc, nivel_agregacion] -> [iso3, anio, pais_nombre, continente_fundar, nivel_agregacion, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio'],
 'CRECIM_g2': ['Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-25, creando la columna niv

In [23]:
exportar(_df)

('../output/CRECIM1/definitivos/CRECIM_g3.csv',
 '../output/CRECIM1/definitivos/CRECIM_g3_old.csv')

SEBACO_g4.csv

In [16]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 4

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,exportaciones_sbc,prop_expo_sbc_servicios,prop_expo_sbc_total
0,2006,2521.420877,0.318723,0.046240
1,2007,3307.066349,0.329201,0.050014
2,2008,4251.379841,0.372140,0.052111
3,2009,4360.837831,0.413560,0.065739
4,2010,5031.582871,0.392567,0.062024


In [17]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    #nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,2006,exportaciones_sbc,2521.420877
1,2006,prop_expo_sbc_servicios,0.318723
2,2006,prop_expo_sbc_total,0.046240
3,2007,prop_expo_sbc_servicios,0.329201
4,2007,prop_expo_sbc_total,0.050014


In [18]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio']}

In [19]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g4.csv',
 '../output/SEBACO1/definitivos/SEBACO_g4_old.csv')

SEBACO_g5.csv

In [20]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 5

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio sbc


,anio,sbc,prop_sbc
0,1996,140069.00,0.039892
1,1997,161740.25,0.042317
2,1998,179796.25,0.044024
3,1999,179947.25,0.044451
4,2000,184365.00,0.046143


In [21]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(['anio']),
    sort(['anio']),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,1996,sbc,140069.000000
1,1996,prop_sbc,0.039892
2,1997,prop_sbc,0.042317
3,1997,sbc,161740.250000
4,1998,sbc,179796.250000


In [23]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio']}

In [24]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g5.csv',
 '../output/SEBACO1/definitivos/SEBACO_g5_old.csv')

SEBACO_g6.csv

In [25]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 6

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

provincia anio


,provincia,anio,prop
0,CABA,1996,0.410583
1,CABA,1997,0.414285
2,CABA,1998,0.416813
3,CABA,1999,0.438703
4,CABA,2000,0.459406


In [26]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,provincia,anio,indicador,valor
0,CABA,1996,prop,0.410583
1,CABA,1997,prop,0.414285
2,CABA,1998,prop,0.416813
3,CABA,1999,prop,0.438703
4,CABA,2000,prop,0.459406


In [27]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio']}

In [28]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g6.csv',
 '../output/SEBACO1/definitivos/SEBACO_g6_old.csv')

SEBACO_g7.csv

In [29]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 7

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

provincia anio


,provincia,anio,prop
0,CABA,1996,0.410583
1,CABA,1997,0.414285
2,CABA,1998,0.416813
3,CABA,1999,0.438703
4,CABA,2000,0.459406


In [30]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys),
    sort(primary_keys)
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,provincia,anio,indicador,valor
0,CABA,1996,prop,0.410583
1,CABA,1997,prop,0.414285
2,CABA,1998,prop,0.416813
3,CABA,1999,prop,0.438703
4,CABA,2000,prop,0.459406


In [31]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [32]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g7.csv',
 '../output/SEBACO1/definitivos/SEBACO_g7_old.csv')

SEBACO_g8.csv

In [33]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 8

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

sector anio


,sector,anio,prop
0,"Otros servicios (empresariales, relacionados c...",2015,0.027914
1,"Otros servicios (empresariales, relacionados c...",2016,0.024359
2,"Otros servicios (empresariales, relacionados c...",2017,0.021508
3,"Otros servicios (empresariales, relacionados c...",2018,0.028338
4,"Otros servicios (empresariales, relacionados c...",2019,0.031180


In [34]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys, value_name="_valor"),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    renombrar({'_valor': 'valor'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,sector,anio,indicador,valor
0,"Otros servicios (empresariales, relacionados c...",2015,prop,0.027914
1,"Otros servicios (empresariales, relacionados c...",2016,prop,0.024359
2,"Otros servicios (empresariales, relacionados c...",2017,prop,0.021508
3,"Otros servicios (empresariales, relacionados c...",2018,prop,0.028338
4,"Otros servicios (empresariales, relacionados c...",2019,prop,0.031180


In [35]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [36]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g8.csv',
 '../output/SEBACO1/definitivos/SEBACO_g8_old.csv')

SEBACO_g9.csv

In [37]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 9

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

rama anio


,rama,anio,prop_rama
0,Investigación y desarrollo,1996,0.018198
1,Investigación y desarrollo,1997,0.016010
2,Investigación y desarrollo,1998,0.015622
3,Investigación y desarrollo,1999,0.015495
4,Investigación y desarrollo,2000,0.015130


In [38]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys + ['pais']),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    nomenclar_paises(code_col='iso3', desc_col="pais", new_col=None), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,rama,anio,indicador,valor
0,Investigación y desarrollo,1996,prop_rama,0.018198
1,Investigación y desarrollo,1997,prop_rama,0.016010
2,Investigación y desarrollo,1998,prop_rama,0.015622
3,Investigación y desarrollo,1999,prop_rama,0.015495
4,Investigación y desarrollo,2000,prop_rama,0.015130


In [39]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [40]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g9.csv',
 '../output/SEBACO1/definitivos/SEBACO_g9_old.csv')

SEBACO_g10.csv

In [41]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 10

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

rama anio


,rama,anio,empleo,sbc_perc,total_perc
0,Investigación y desarrollo,1996,2549.00,0.018198,0.000726
1,Investigación y desarrollo,1997,2589.50,0.016010,0.000677
2,Investigación y desarrollo,1998,2808.75,0.015622,0.000688
3,Investigación y desarrollo,1999,2788.25,0.015495,0.000689
4,Investigación y desarrollo,2000,2789.50,0.015130,0.000698


In [42]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(['iso3','pais','anio','apertura_sexo'], value_name="_valor"),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    renombrar({'_valor': 'valor'}),
    nomenclar_paises(code_col='iso3', new_col=None, desc_col=None), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,rama,anio,indicador,valor
0,Investigación y desarrollo,1996,empleo,2549.000000
1,Investigación y desarrollo,1996,sbc_perc,0.018198
2,Investigación y desarrollo,1996,total_perc,0.000726
3,Investigación y desarrollo,1997,empleo,2589.500000
4,Investigación y desarrollo,1997,sbc_perc,0.016010


In [43]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [44]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g10.csv',
 '../output/SEBACO1/definitivos/SEBACO_g10_old.csv')

SEBACO_g11.csv

In [45]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 11

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

sector estado_ocupacional


,sector,estado_ocupacional,prop_ocupados
0,"a. Actividades profesionales, cientificas y té...",Asalariado no registrado,0.146243
1,"a. Actividades profesionales, cientificas y té...",Asalariado registrado,0.324780
2,"a. Actividades profesionales, cientificas y té...",No asalariado,0.528977
3,b. SSI,Asalariado no registrado,0.105757
4,b. SSI,Asalariado registrado,0.676609


In [52]:
# m49_cod = paises[['m49_code_unsd', 'iso3_desc_fundar']].dropna(how="any").sort_values(by="m49_code_unsd").reset_index(drop=True)
# m49_cod['m49_code_unsd'] = m49_cod['m49_code_unsd'].astype('int')
# pais_m49_cod = {k:v for k,v in m49_cod.iloc}
# codes_set_m49_cod = set(pais_m49_cod.keys())


In [46]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    nomenclar_paises(code_col='iso3', new_col=None, desc_col="pais_desc"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,sector,estado_ocupacional,indicador,valor
0,"a. Actividades profesionales, cientificas y té...",Asalariado no registrado,prop_ocupados,0.146243
1,"a. Actividades profesionales, cientificas y té...",Asalariado registrado,prop_ocupados,0.324780
2,"a. Actividades profesionales, cientificas y té...",No asalariado,prop_ocupados,0.528977
3,b. SSI,Asalariado no registrado,prop_ocupados,0.105757
4,b. SSI,Asalariado registrado,prop_ocupados,0.676609


In [47]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [48]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g11.csv',
 '../output/SEBACO1/definitivos/SEBACO_g11_old.csv')

SEBACO_g12.csv

In [49]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 12

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio sector_sbc


,anio,sector_sbc,salario_promedio
0,1996,Investigación y desarrollo,540329.254131
1,1996,Promedio economía,396409.568514
2,1996,SBC Otras,342582.840711
3,1996,SSI,882113.180802
4,1996,Ss. Arquitectura,385331.029244


In [50]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,sector_sbc,indicador,valor
0,1996,Investigación y desarrollo,salario_promedio,540329.254131
1,1996,Promedio economía,salario_promedio,396409.568514
2,1996,SBC,salario_promedio,416692.466719
3,1996,SBC Otras,salario_promedio,342582.840711
4,1996,SSI,salario_promedio,882113.180802


In [51]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [52]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g12.csv',
 '../output/SEBACO1/definitivos/SEBACO_g12_old.csv')

SEBACO_g13.csv

In [53]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 13

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

nivel sector


,nivel,sector,prop_educ
0,a. Primario incompleto,SBC,0.001950
1,a. Primario incompleto,Sector privado,0.032459
2,a. Primario incompleto,Total economía,0.028381
3,b. Primario completo,SBC,0.005727
4,b. Primario completo,Sector privado,0.128996


In [54]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,nivel,sector,indicador,valor
0,a. Primario incompleto,SBC,prop_educ,0.001950
1,a. Primario incompleto,Sector privado,prop_educ,0.032459
2,a. Primario incompleto,Total economía,prop_educ,0.028381
3,b. Primario completo,SBC,prop_educ,0.005727
4,b. Primario completo,Sector privado,prop_educ,0.128996


In [55]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [56]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g13.csv',
 '../output/SEBACO1/definitivos/SEBACO_g13_old.csv')

SEBACO_g14.csv

In [57]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 14

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio sector


,anio,sector,prop_mujeres
0,2007,SBC,0.384661
1,2007,Total economia,0.301126
2,2008,SBC,0.388556
3,2008,Total economia,0.306189
4,2009,SBC,0.401199


In [58]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,sector,indicador,valor
0,2007,SBC,prop_mujeres,0.384661
1,2007,Total economia,prop_mujeres,0.301126
2,2008,SBC,prop_mujeres,0.388556
3,2008,Total economia,prop_mujeres,0.306189
4,2009,SBC,prop_mujeres,0.401199


In [59]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [60]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g14.csv',
 '../output/SEBACO1/definitivos/SEBACO_g14_old.csv')

SEBACO_g15.csv

In [61]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 15

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio sector


,anio,sector,prop_mujeres
0,1996,Promedio economia,0.267976
1,1997,Promedio economia,0.271105
2,1998,Promedio economia,0.282673
3,1999,Promedio economia,0.288951
4,2000,Promedio economia,0.299164


In [63]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,sector,indicador,valor
0,1996,Promedio economia,prop_mujeres,0.267976
1,1996,SSI,prop_mujeres,0.329125
2,1997,Promedio economia,prop_mujeres,0.271105
3,1997,SSI,prop_mujeres,0.332967
4,1998,Promedio economia,prop_mujeres,0.282673


In [64]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [65]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g15.csv',
 '../output/SEBACO1/definitivos/SEBACO_g15_old.csv')

SEBACO_g16.csv

In [66]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 16

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

periodo sector


,periodo,brecha,sector
0,1996,0.237825,Promedio economía
1,1997,0.226060,Promedio economía
2,1998,0.226689,Promedio economía
3,1999,0.219682,Promedio economía
4,2000,0.224453,Promedio economía


In [67]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,periodo,sector,indicador,valor
0,1996,Promedio economía,brecha,0.237825
1,1996,SSI,brecha,0.399256
2,1997,Promedio economía,brecha,0.226060
3,1997,SSI,brecha,0.380554
4,1998,Promedio economía,brecha,0.226689


In [68]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [69]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g16.csv',
 '../output/SEBACO1/definitivos/SEBACO_g16_old.csv')

SEBACO_g17.csv

In [70]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 17

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio sector


,anio,sector,prop_teletrabajo
0,2016,SBC,0.113657
1,2016,SSI,0.117739
2,2016,Total,0.010428
3,2017,SBC,0.138398
4,2017,SSI,0.151250


In [71]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,sector,indicador,valor
0,2016,SBC,prop_teletrabajo,0.113657
1,2016,SSI,prop_teletrabajo,0.117739
2,2016,Total,prop_teletrabajo,0.010428
3,2017,SBC,prop_teletrabajo,0.138398
4,2017,SSI,prop_teletrabajo,0.151250


In [72]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [73]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g17.csv',
 '../output/SEBACO1/definitivos/SEBACO_g17_old.csv')

SEBACO_g18.csv

In [74]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 18

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio sector


,anio,sector,balanza,exportaciones,importaciones
0,2006,SBC,200.396386,2521.420877,2321.024492
1,2007,SBC,359.301850,3307.066349,2947.764499
2,2008,SBC,475.756289,4251.379841,3775.623552
3,2009,SBC,403.170159,4360.837831,3957.667671
4,2010,SBC,529.793094,5031.582871,4501.789776


In [75]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,sector,indicador,valor
0,2006,Investigación y desarrollo,balanza,106.637260
1,2006,Investigación y desarrollo,exportaciones,123.311501
2,2006,Investigación y desarrollo,importaciones,16.674241
3,2006,Propiedad intelectual,balanza,-814.311699
4,2006,Propiedad intelectual,exportaciones,80.512423


In [76]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [77]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g18.csv',
 '../output/SEBACO1/definitivos/SEBACO_g18_old.csv')

SEBACO_g12.csv

In [78]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 19

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio sector


,anio,sector,balanza
0,2006,Propiedad intelectual,-814.311699
1,2007,Propiedad intelectual,-1077.988222
2,2008,Propiedad intelectual,-1432.359077
3,2009,Propiedad intelectual,-1420.929871
4,2010,Propiedad intelectual,-1560.024589


In [79]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,sector,indicador,valor
0,2006,Investigación y desarrollo,balanza,106.637260
1,2006,Propiedad intelectual,balanza,-814.311699
2,2006,SSI,balanza,121.261315
3,2006,Servicios profesionales,balanza,479.618757
4,2006,"Ss. arquitectura, ingeniería y otros",balanza,226.783660


In [80]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [81]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g19.csv',
 '../output/SEBACO1/definitivos/SEBACO_g19_old.csv')

SEBACO_g20.csv

In [82]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 20

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio descripcion


,anio,descripcion,prop_sobre_sbc_expo
0,2006,Propiedad intelectual,0.031931
1,2007,Propiedad intelectual,0.029239
2,2008,Propiedad intelectual,0.023373
3,2009,Propiedad intelectual,0.024299
4,2010,Propiedad intelectual,0.030247


In [83]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,descripcion,indicador,valor
0,2006,Investigación y desarrollo,prop_sobre_sbc_expo,0.048906
1,2006,Propiedad intelectual,prop_sobre_sbc_expo,0.031931
2,2006,SSI,prop_sobre_sbc_expo,0.134202
3,2006,Servicios profesionales,prop_sobre_sbc_expo,0.389265
4,2006,"Ss. arquitectura, ingeniería y otros",prop_sobre_sbc_expo,0.314426


In [84]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [85]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g20.csv',
 '../output/SEBACO1/definitivos/SEBACO_g20_old.csv')

SEBACO_g21.csv

In [86]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 21

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,exportaciones,prop_sobre_expo_total
0,2006,338.380322,0.006206
1,2007,551.612442,0.008342
2,2008,816.851418,0.010012
3,2009,929.182930,0.014007
4,2010,1142.671626,0.014086


In [87]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,2006,exportaciones,338.380322
1,2006,prop_sobre_expo_total,0.006206
2,2007,prop_sobre_expo_total,0.008342
3,2007,exportaciones,551.612442
4,2008,exportaciones,816.851418


In [88]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [89]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g21.csv',
 '../output/SEBACO1/definitivos/SEBACO_g21_old.csv')

SEBACO_g22.csv

In [90]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 22

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,iso3,anio,prop_expo
0,AFG,2008,0.000248
1,AFG,2009,0.000478
2,AFG,2010,0.000413
3,AFG,2011,0.000407
4,AFG,2012,0.000258


In [91]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col=None),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,indicador,valor
0,ABW,2005,prop_expo,0.000146
1,ABW,2006,prop_expo,0.000158
2,ABW,2007,prop_expo,0.000129
3,ABW,2008,prop_expo,0.000111
4,ABW,2009,prop_expo,0.000160


In [92]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [93]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g22.csv',
 '../output/SEBACO1/definitivos/SEBACO_g22_old.csv')

SEBACO_g23.csv

In [94]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 23

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,iso3,anio,prop_expo
0,AFG,2008,0.000096
1,AFG,2009,0.000095
2,AFG,2010,0.000019
3,AFG,2011,0.000014
4,AFG,2012,0.000048


In [95]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col=None),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,indicador,valor
0,AFG,2008,prop_expo,0.000096
1,AFG,2009,prop_expo,0.000095
2,AFG,2010,prop_expo,0.000019
3,AFG,2011,prop_expo,0.000014
4,AFG,2012,prop_expo,0.000048


In [96]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [97]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g23.csv',
 '../output/SEBACO1/definitivos/SEBACO_g23_old.csv')

SEBACO_g24.csv

In [98]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 24

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio iso3


,anio,iso3,expo
0,2017,NAM,0.0
1,2019,NAM,0.0
2,2021,NAM,0.0
3,2022,NAM,0.0
4,2015,AND,0.1


In [99]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col=None),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,iso3,indicador,valor
0,2015,ABW,expo,0.1
1,2015,AGO,expo,0.0
2,2015,AIA,expo,0.0
3,2015,ALB,expo,0.0
4,2015,AND,expo,0.1


In [100]:
diccionario_cambios

{'SEBACO_g1': ['Long estricto: [anio, sector, share_ventas] -> [anio, sector, valor, indicador]',
  'Ordené las filas por las columnas: anio, sector'],
 'SEBACO_g2': ['Long estricto: [anio, vbp_ssi_2022, prop_ssi_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las 

In [101]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g24.csv',
 '../output/SEBACO1/definitivos/SEBACO_g24_old.csv')

SEBACO_g25.csv

In [102]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 25

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio iso3


,anio,iso3,impo
0,2015,NAM,0.0
1,2015,AND,0.0
2,2016,AND,0.0
3,2017,AND,0.0
4,2018,AND,0.0


In [103]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col=None),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,iso3,indicador,valor
0,2015,ABW,impo,0.0
1,2015,AGO,impo,0.0
2,2015,ALB,impo,0.8
3,2015,AND,impo,0.0
4,2015,ARE,impo,5.0


In [104]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
 'SEBACO_g3': ['Long estricto: [anio, vab] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g4': ['Long estricto: [anio, exportaciones_sbc, prop_expo_sbc_servicios, prop_expo_sbc_total] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g5': ['Long estricto: [anio, sbc, prop_sbc] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'SEBACO_g6': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g7': ['Long estricto: [provincia, anio, prop] -> [provincia, anio, valor, indicador]',
  'Ordené las filas por las columnas: provincia, anio'],
 'SEBACO_g8'

In [105]:
exportar(_df)

('../output/SEBACO1/definitivos/SEBACO_g25.csv',
 '../output/SEBACO1/definitivos/SEBACO_g25_old.csv')

In [107]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
match_relocate(f'{folder}/definitivos', f'{folder}/old', '.*old.*')

Files matching the pattern 're.compile('.*old.*')' moved to '../output/SEBACO1/old'.


In [108]:
import json
import datetime

today = datetime.datetime.now().strftime("%Y-%m-%d")

with open(f'../output/SEBACO1/definitivos/paso_a_definitivos_{today}.json', 'w') as fp:
    json.dump(diccionario_cambios, fp, indent=4, ensure_ascii=False)
